In [1]:
# Install required packages (if not installed)
!pip install chromadb pandas sentence-transformers transformers

# Imports
import pandas as pd
import chromadb
from sentence_transformers import SentenceTransformer
from chromadb.config import Settings
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

# === Load CSV Data ===
filepath = '/content/pace_graduate_programs (1).csv'  # <- Make sure you upload the file
df = pd.read_csv(filepath)

program_names = df['Program Name'].tolist()
program_links = df['Program Link'].tolist()

# === Embed Program Names ===
model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(program_names)

# === Initialize Chroma Vector Store ===
chroma_client = chromadb.Client(Settings(anonymized_telemetry=False))
collection = chroma_client.create_collection(name="pace_programs")

# Add documents with metadata (links)
collection.add(
    documents=program_names,
    metadatas=[{"link": link} for link in program_links],
    embeddings=embeddings.tolist(),
    ids=[f"id{i}" for i in range(len(program_names))]
)

# === Semantic Search Function ===
def search_programs(user_query, embedding_model, top_k=5):
    query_embed = embedding_model.encode([user_query])[0].tolist()
    results = collection.query(
        query_embeddings=[query_embed],
        n_results=top_k
    )
    programs = results['documents'][0]
    metadata = results['metadatas'][0]
    return list(zip(programs, [meta['link'] for meta in metadata]))

# === Load RAG Model ===
model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
llm_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
rag_pipeline = pipeline("text2text-generation", model=llm_model, tokenizer=tokenizer)

# === RAG Answer Generator ===
def generate_answer_with_flan(query, program_tuples):
    context = "\n".join([f"{name} – {link}" for name, link in program_tuples])
    prompt = f"""You are a helpful assistant that answers questions about graduate programs at Pace University.

Programs:
{context}

Question: {query}

Answer:"""
    result = rag_pipeline(prompt, max_new_tokens=150)[0]['generated_text']
    return result

# === Greeting and Exit Helpers ===
def is_exit(text):
    return text.strip().lower() in ['exit', 'quit', 'bye', 'see you later']

def is_greeting(text):
    return any(greet in text.lower() for greet in ['hi', 'hello', 'hey'])

# === Chatbot Interface ===
print("🎓 PaceBot: Hello! I'm PaceBot, your assistant for graduate programs at Pace University.")
print("🎓 Ask me anything like 'Do you offer data science?' or 'Tell me about MBA programs'. Type 'exit' to leave.\n")

while True:
    user_input = input("You: ")

    if is_exit(user_input):
        print("PaceBot: Chat with you later! 🎓")
        break
    elif is_greeting(user_input):
        print("PaceBot: Hello! How can I help you today?")
    else:
        program_matches = search_programs(user_input, model)
        response = generate_answer_with_flan(user_input, program_matches)
        print("PaceBot:", response)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 7.4 MB/s eta 0

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cpu


🎓 PaceBot: Hello! I'm PaceBot, your assistant for graduate programs at Pace University.
🎓 Ask me anything like 'Do you offer data science?' or 'Tell me about MBA programs'. Type 'exit' to leave.

You: hie
PaceBot: Hello! How can I help you today?
You: what programs are there at pace
PaceBot: Pace University-Lenox Hill Hospital Physician Assistant Program, MS – https://www.pace.edu/program/pace-university-lenox-hill-hospital-physician-assistant-program-ms Physician Assistant Master's Degree Completion Program, MS – https://www.pace.edu/program/physician-assistant-masters-degree-completion-program-ms Physician Assistant Program-Pleasantville, MS – https://www.pace.edu/program/doctor-of-professional-studies-executive-doctoral
You: is there data science at pace
PaceBot: no
You: is there computer science
PaceBot: yes
You: what about data science
PaceBot: Computer Science, PhD – https://www.pace.edu/program/computer-science-phd Computer Science, MS – https://www.pace.edu/graduate-programs/ms